## KoGPT2
 Day04-1104 ~ Day06-1106

* MODEL3 : SKT koGPT 2
 - Base Model(110M) 까지는 LMS 환경에서 사용 가능하다고 들어 적합한 사이즈의 모델을 서칭하여 선정
 - dataset 구성 : 최종  train 14000, val 3500, test 3500 참고 할것

 - [Huggingface_GPT2](https://huggingface.co/docs/transformers/model_doc/gpt2)
 - [SKT-koGPT2](https://github.com/SKT-AI/KoGPT2#model)
 - [GPT2_finetuning](https://hyyoka-ling-nlp.tistory.com/9)
 - [GPT2_finetuning_NonEng](https://www.philschmid.de/fine-tune-a-non-english-gpt-2-model-with-huggingface)
 - [GPT2 text summarization finetuning](https://towardsdatascience.com/text-summarization-with-gpt2-and-layer-ai-599625085d8e)

* 추상요약
 - [Abstractive Summarization](https://medium.com/search?q=abstractive+text+summarization)


 GPT2 모델 학습
  1. Data 로드
   * 데이터 형식 변경 : train/validation/ test 하나로 취합하여 로드  
   * 데이터 구성 : train/validation/test 형식에 맞추어 변경 후 데이터 다운로드)

2. GPT Tokenizer, Model 허깅페이스 로드 
 * 4단계.trainer에서 dimension mismatch 오류가 지속적으로 발생하여 학습불가 
    * 2단계.토크나이징 단계에 동적 패딩 적용하여 디멘젼 오류 해결

3. Load Train Config 
 * 5.test dataset evaluation단계 까지 완료 한 후 다시 3.Load Train Config 단계로 돌아와 파인튜닝하여 성능 향상 최종 목표!)

4. Trainer : 
 * traindatat 학습 예상 시간이 너무 길어서 학습 불가
    * max step size 설정하여 train data학습양과 시간 줄여서 학습 성공
 * validation dataset의 크기가 너무 커서 현재 trainer 완료 하지 못한 상태
    * 해결책 찾는중
    
5. Test evaluation 

In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# PROJECT : 뉴스 데이터셋 요약
* Task : summary

In [3]:
import os
import numpy as np
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures

## (1) Dataset Load

>Dataset dictionary
* train dataset, validation dataset, test dataset으로 구성
* Dataset은 ‘sentence1’, ‘sentence2’, ‘label’, ‘idx’(인덱스)로 구성 

 Huggingface datasets를 사용하면 DataProcessor를 사용할 필요없이 바로 사용가능하게 구성되어 있습니다.

In [4]:
# import datasets
# from datasets import load_dataset

# huggingface_mrpc_dataset = load_dataset('glue', 'mrpc')
# print(huggingface_mrpc_dataset)

>News Dataset dictionary
* train dataset, validation dataset 으로 구성
* Dataset은 ‘input’, target'로 구성 

In [5]:
# Load a CSV file
from datasets import load_dataset
# train_Sum1 = load_dataset('csv', data_files='/content/drive/MyDrive/Jiwon/Data/train_Sum1.csv')
# val_Sum1 = load_dataset('csv', data_files='/content/drive/MyDrive/Jiwon/Data/val_Sum1.csv')


* 데이터 로드할때 형식 맞춰주기 
```
data_files = {"train": "/content/drive/MyDrive/Jiwon/Data/train_Sum1.csv", "test": "/content/drive/MyDrive/Jiwon/Data/val_Sum1.csv"}
dataset = load_dataset('csv', data_files=data_files)
```



In [6]:
data_files = {
    "train" : "/content/drive/MyDrive/Jiwon/Data/train_Sum1_.csv",
    "val"   : "/content/drive/MyDrive/Jiwon/Data/val_Sum1.csv",
    "test"  : "/content/drive/MyDrive/Jiwon/Data/test_Sum1.csv"
    }
dataset = load_dataset('csv', data_files=data_files)

  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 15250
    })
    val: Dataset({
        features: ['input', 'target'],
        num_rows: 18300
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 3050
    })
})

In [8]:
# import pandas as pd

# train_Sum1 = pd.read_csv("/content/drive/MyDrive/Jiwon/Data/train_Sum1.csv")
# val_Sum1 = pd.read_csv("/content/drive/MyDrive/Jiwon/Data/val_Sum1.csv")

In [9]:
# train_Sum1.head()

In [10]:
# val_Sum1.head()

## (2) Tokenizer, Model

> Model, Tokenizer 생성
* Huggingface- Auto Class
 - Auto class는 from_pretrained 메소드를 이용해 pretrained model의 경로 혹은 이름만 안다면 자동으로 생성하는 방법
 - AutoTokenizer, AutoModel기능 지원



model = AutoModelForCausalLM.from_pretrained

In [11]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("skt/kogpt2-base-v2",
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')

In [12]:
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [13]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                          max_length=128,
                          repetition_penalty=2.0,
                          pad_token_id=tokenizer.pad_token_id,
                          eos_token_id=tokenizer.eos_token_id,
                          bos_token_id=tokenizer.bos_token_id,
                          use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


> 토크나이징
* `transform()` 함수를 만들고 이전에 만들어두었던 Tokenizer에서 입력할 `dataset의 형태를 확인하고 바꿀 대상을 지정`해야 합니다.

In [14]:
def transform(data):
  return tokenizer(
      text = data['input'],
      text_target = data['target'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = False,
      max_length=128
      )
  
examples = dataset['train'][:5]
examples_transformed = transform(examples)

print(examples)
print(examples_transformed)

{'input': ['4일 오후 4시 충남 아산시 신리초등학교 체육관 2층 입구.\n  손 소독과 발열 체크를 한 뒤 체육관 안으로 들어가라는 안내문이 붙었다.\n  마스크를 착용하지 않으면 출입이 통제됐다.\n  체육관으로 들어가자 왼쪽에 설치된 대형 칠판에 ‘사람 두 팔 간격 건강 거리 유지’라는 글이 크게 쓰여 있었다.\n  지난 3일 충남 아산지역에 폭우가 내리면서 온양천이 범람하자 신동·모종동 주민들이 급히 신리초로 대피했다.\n  신리초는 홍수와 지진 등 각종 재난사고 때 임시 주거시설로 지정된 곳이다.\n  3일 오후 5시까지 60여 명 수준이던 이재민은 오후 7시가 지나서는 100여 명을 넘어섰다.\n  아산시는 주민들이 한꺼번에 몰리자 수해와 더불어 신종 코로나바이러스 감염증(코로나19)이 확산할 것을 우려했다.\n  가장 먼저 주민들에게 마스크 착용을 의무화했다.\n  마스크를 쓰지 않으면 체육관 출입을 제한하고 체육관에 머무는 시간에도 밥을 먹거나 물을 마실 때를 제외하고 마스크를 벗지 못하게 했다.\n  이재민들은 집이나 농경지로 오갈 때도 출입 횟수에 관계없이 매번 발열 체크를 했다.\n  외부에서 다른 사람과 접촉할 가능성이 있어서다.\n  4일 오후 4시 30분쯤 체육관 안에는 주민 5명과 아산시청 공무원·자원봉사자 10여 명이 머물고 있었다.\n  주민들은 아산시가 제공한 매트리스 위에서 쉬거나 대화를 나눴다.\n  매트리스 1개마다 1명씩 배정됐다.\n  매트리스 간격은 2~3m가량으로 1개의 매트리스에 2명 이상이 앉지 못하도록 했다.\n  주민 간 접촉을 최소화하기 위해서다.\n  다른 대피소와 달리 신리초 체육관에는 소형 천막(텐트)이 지급되지 않았다.\n  이 때문에 충남도와 아산시 등 방역 당국이 바짝 긴장했다.\n  이곳에 머무르는 주민들은 대부분 70~80대 고령자로 코로나19에 노출될 경우 감염 위험이 높아서다.\n  아산시는 주민들이 식사를 할 때 밀접 접촉할 가능성이 크다고 판단, 모든 식사를 도시락으로 대체했다

In [15]:
# import numpy as np
# len(encoded_data['test']['input_ids'][0]), len(encoded_data['train']['input_ids'][0])

In [16]:
# len(encoded_data['test']['labels'][0]), len(encoded_data['train']['labels'][0])

In [17]:
# len(encoded_data['test']['attention_mask'][0]), len(encoded_data['train']['attention_mask'][0])

expected sequence of length 278 at dim 1 (got 281)

In [18]:
# import seaborn as sns 

> `map():`
* Data dictionary에 있는 모든 데이터들이 한번에 토크나징 적용시킬 수 있습니다.
* 우리는 map을 사용해 토크나이징을 진행하기 때문에 batch를 적용해야 되므로 batched=True 설정

In [19]:
encoded_data = dataset.map(transform, batched=True)

In [20]:
encoded_data

DatasetDict({
    train: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15250
    })
    val: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 18300
    })
    test: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3050
    })
})

## (3) Train/Evaluation과 Test

>Trainer
* 사용하기 위해서는 TrainingArguments를 통해 학습 관련 설정을 미리 지정

In [21]:
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
! pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import evaluate

In [24]:
rouge = evaluate.load('rouge')

predictions = ["hello there 2", "general kenobi"]
references = ["hello there", "general kenobi"]
results = rouge.compute(predictions=predictions, references=references)
print(results)

{'rouge1': 0.9, 'rouge2': 0.8333333333333333, 'rougeL': 0.9, 'rougeLsum': 0.9}


In [25]:
# def _rouge(guess, answers):
#     global rouge
#     """Compute ROUGE score between guess and *any* answers. Return the best."""
#     if rouge is None:
#         return None, None, None
#     evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l'], max_n=2)
#     try:
#         scores = [
#             evaluator.get_scores(normalize_answer(guess), normalize_answer(a))
#             for a in answers
#         ]
#     except LookupError:
#         warn_once(
#             'ROUGE requires nltk punkt tokenizer. Please run '
#             '`python -c "import nltk; nltk.download(\'punkt\')`'
#         )
#         rouge = None
#         return None, None, None

#     scores_rouge1 = [score['rouge-1']['r'] for score in scores]
#     scores_rouge2 = [score['rouge-2']['r'] for score in scores]
#     scores_rougeL = [score['rouge-l']['r'] for score in scores]
#     return max(scores_rouge1), max(scores_rouge2), max(scores_rougeL)

In [26]:
# scores = rouge.compute(
#     predictions=[generated_summary], references=[reference_summary]
# )
# scores

In [27]:
def compute_rouge(eval_pred):    
    predictions,references = eval_pred
    return rouge.compute(predictions=predictions, references = references)

* max_steps = 3

In [28]:
# Trainer을 활용하는 형태로 모델 재생성
from transformers import Trainer, TrainingArguments
output_dir = '/content/drive/MyDrive/Jiwon/transformers'
metric_name = 'rouge'

training_arguments = TrainingArguments(
    output_dir, # output이 저장될 경로
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, #learning_rate
    per_device_train_batch_size = 16, # 각 device 당 batch size
    per_device_eval_batch_size = 16, # evaluation 시에 batch size
    num_train_epochs = 3, # train 시킬 총 epochs
    weight_decay = 0.01, # weight decay
    max_steps = 3,
    do_eval = False
)

In [29]:
from datasets import load_metric
# metric = load_metric('glue', 'mnli')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

In [30]:
trainer = Trainer(
    model=model,                           # 학습시킬 model
    args=training_arguments,                  # TrainingArguments을 통해 설정한 arguments
    train_dataset=encoded_data['train'],    # training dataset
    eval_dataset=encoded_data['test'],       # evaluation dataset
    compute_metrics=compute_rouge,
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: target, input. If target, input are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 15250
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3
  Number of trainable parameters = 125164032


Epoch,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: target, input. If target, input are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3050
  Batch size = 16


In [ ]:
# trainer.evaluate(encoded_dataset['test'])

In [ ]:
 rouge.compute(encoded_data[test])